In [1]:
# if use google colab, mount your drive

# from google.colab import drive
# drive.mount('/content/gdrive')

# dataset_path = F"/content/gdrive/My Drive/dsai_final/"

In [1]:
import glob
import os

In [2]:
glob.glob(os.path.join('..', '*.csv'))

['../uxp_full_data.csv', '../test.csv', '../uxp_10_percent_data.csv']

# Read Training Data

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [4]:
# read training data
df = pd.read_csv('../uxp_full_data.csv', index_col=0)

/home/jeff/anaconda3/envs/work/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
# get number of user and product
N_USER = np.max(df['user_id'])
N_PRODUCT = np.max(df['product_id'])

N_USER, N_PRODUCT

(206209, 49688)

In [6]:
df.head()

,user_id,product_id,uxp_reorder_ratio,u_total_bought_product,u_avg_bought_p_per_order,uxp_order_ratio,uxp_bought_before,score
0,1,196,1.000000,70,6.363636,1.000000,1,2.100000
1,1,10258,1.000000,70,6.363636,0.909091,1,2.009091
2,1,10326,0.142857,70,6.363636,0.090909,1,0.333766
3,1,12427,0.909091,70,6.363636,0.909091,1,1.918182
4,1,13032,0.400000,70,6.363636,0.363636,1,0.863636


In [7]:
df.tail()

,user_id,product_id,uxp_reorder_ratio,u_total_bought_product,u_avg_bought_p_per_order,uxp_order_ratio,uxp_bought_before,score
13863741,206209,43961,0.272727,137,9.785714,0.214286,1,0.587013
13863742,206209,44325,0.125000,137,9.785714,0.071429,1,0.296429
13863743,206209,48370,0.250000,137,9.785714,0.071429,1,0.421429
13863744,206209,48697,0.125000,137,9.785714,0.071429,1,0.296429
13863745,206209,48742,0.250000,137,9.785714,0.142857,1,0.492857


# Create LeaveOneOut for each user

{user_id : product_id}

Efficient way to leaveOneOut

In [8]:
loo = df.groupby('user_id')['product_id'].max().to_frame('product_id')

loo = loo.reset_index()

loo.head()

,user_id,product_id
0,1,49235
1,2,49451
2,3,49683
3,4,43704
4,5,48775


In [9]:
leaveOneOut = {}

for i in range(len(loo)):
    leaveOneOut[loo['user_id'].iloc[i]] = loo['product_id'].iloc[i]

Don't Use This so slow...

In [10]:
# leaveOneOut = {}

# # loop through all user who had bought
# for i in tqdm(np.unique(df['user_id'])):
#   # get last element of user bought product list
#   leaveOneOut[i] = df[df['user_id'] == i]['product_id'].to_numpy()[-1]

In [11]:
# get first 5 user_id:product_id pair
list(leaveOneOut.items())[:5]

[(1, 49235), (2, 49451), (3, 49683), (4, 43704), (5, 48775)]

# Process Training Data

## Remove leaveOneOut product from train set

In [12]:
trainset = df.copy()

In [13]:
f = open('leaveOneOutRows.txt')

drop_rows = f.readlines()

drop_rows = [int(i[:-1]) for i in drop_rows]

# drop_rows

In [14]:
# drop the last product userId[i] bought from the training set
# very slow

# drop_rows = []

# for i in tqdm(leaveOneOut.keys()):
#   query0 = trainset[trainset['user_id'] == i]
#   query1 = query0[query0['product_id'] == leaveOneOut[i]]

#   drop_rows.append(list(query1.index)[0])

trainset = trainset.drop(drop_rows)

In [15]:
# testset

testset = df.copy()

testset = testset.loc[df.index.isin(drop_rows)] 

testset.head()

,user_id,product_id,uxp_reorder_ratio,u_total_bought_product,u_avg_bought_p_per_order,uxp_order_ratio,uxp_bought_before,score
18,1,49235,0.750000,70,6.363636,0.272727,1,1.122727
139,2,49451,0.083333,226,15.066667,0.066667,1,0.250000
172,3,49683,0.100000,88,7.333333,0.083333,1,0.283333
189,4,43704,0.333333,18,3.600000,0.200000,1,0.633333
217,5,48775,0.200000,46,9.200000,0.200000,1,0.500000


In [16]:
# write drop rows to txt, because it is soooo slow
# f = open('leaveOneOutRows.txt', 'w')

# for r in drop_rows:
#     f.write("{}\n".format(r))

# f.close()

In [16]:
# sanity check
# see if trainset is indeed shorter than origin df

len(df), len(trainset), len(testset)

(13863746, 13657537, 206209)

In [17]:
trainset.head()

,user_id,product_id,uxp_reorder_ratio,u_total_bought_product,u_avg_bought_p_per_order,uxp_order_ratio,uxp_bought_before,score
0,1,196,1.000000,70,6.363636,1.000000,1,2.100000
1,1,10258,1.000000,70,6.363636,0.909091,1,2.009091
2,1,10326,0.142857,70,6.363636,0.090909,1,0.333766
3,1,12427,0.909091,70,6.363636,0.909091,1,1.918182
4,1,13032,0.400000,70,6.363636,0.363636,1,0.863636


## Shuffle Trainset

In [18]:
from sklearn.utils import shuffle

In [19]:
trainset = shuffle(trainset)

In [20]:
trainset.head()

,user_id,product_id,uxp_reorder_ratio,u_total_bought_product,u_avg_bought_p_per_order,uxp_order_ratio,uxp_bought_before,score
10290184,152911,38033,1.000000,174,15.818182,0.181818,1,1.281818
13407863,199371,44628,0.100000,333,23.785714,0.071429,1,0.271429
2185754,32660,37004,0.333333,524,9.192982,0.017544,1,0.450877
10088397,149979,36443,0.333333,96,16.000000,0.166667,1,0.600000
1529985,22880,10246,0.044444,696,13.132075,0.037736,1,0.182180


## define dataset

In [21]:
from torch.utils.data import Dataset

In [22]:
class UXP(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        user = self.df.iloc[idx].user_id
        product = self.df.iloc[idx].product_id
        y = self.df.iloc[idx].score

        return np.expand_dims(user, 0).astype(int), np.expand_dims(product, 0).astype(int), np.expand_dims(y, 0).astype(np.float32)

In [23]:
dataset = UXP(trainset)

In [24]:
len(dataset)

13657537

In [25]:
dataset[0]

(array([152911]), array([38033]), array([1.2818182], dtype=float32))

In [26]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset, batch_size=4096, shuffle=True, num_workers=2)

# define model

we try basic Matrix Factorization here

In [27]:
import torch
import torch.nn as nn

In [28]:
torch.cuda.is_available()

True

In [29]:
class CFMF(nn.Module):
  def __init__(self, embed_dim, num_user, num_product):
    super().__init__()
    self.user_embedding = nn.Embedding(num_user + 1, embed_dim)
    self.product_embedding = nn.Embedding(num_product + 1, embed_dim)

  def forward(self, u, p):
    u_emb = self.user_embedding(u)
    p_emb = self.product_embedding(p)

    # move p_emb from (b, 1, emb_dim) to (b, emb_dim, 1)
    # in order to do matmul
    p_emb = torch.movedim(p_emb, 1, 2)

    # dot of user embedding and product embedding
    # u_emb (b, 1, emb_dim)
    # p_emb (b, emb_dim, 1)
    mat = torch.bmm(u_emb, p_emb)

    return mat

In [30]:
model = CFMF(10, N_USER, N_PRODUCT)

In [31]:
if torch.cuda.is_available():
  model.to('cuda')

## test fit model

In [32]:
!nvidia-smi

Wed Jun  9 20:07:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:01:00.0  On |                  N/A |
| 43%   39C    P2    29W / 120W |   1261MiB /  3016MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [33]:
for u, p, y in train_loader:
  with torch.no_grad():
    print(u.shape, p.shape, y.shape)
    if torch.cuda.is_available():
      u = u.cuda()
      p = p.cuda()

    mat = model(u, p)
    print(mat.shape)
    print(mat.squeeze())
  break

torch.Size([4096, 1]) torch.Size([4096, 1]) torch.Size([4096, 1])
torch.Size([4096, 1, 1])
tensor([-0.1959,  1.6157, -0.3976,  ...,  0.2133,  0.2827, -1.8660],
       device='cuda:0')


# Training

## define loss and optimizer

In [34]:
loss = nn.MSELoss()

In [41]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

## training loop

In [42]:
for epoch in range(100):
  # collect loss for a epoch
  loss_collect = []

  for u, p, y in tqdm(train_loader):
    # check if gpu avaliable
    if torch.cuda.is_available():
      u = u.to('cuda')
      p = p.to('cuda')
      y = y.to('cuda')

    # forward
    pr = model(u, p)

    # calculate loss
    _loss = loss(pr.squeeze(), y.squeeze())

    # backward
    _loss.backward()
    optimizer.step()

    # track loss for each batch
    loss_collect.append(_loss.cpu().item())
  
  # print loss tracking
  print("Epoch: {} : {}".format(epoch, np.mean(loss_collect)))

  0%|          | 0/3335 [00:00<?, ?it/s]

Epoch: 0 : 6.066217523786439


  0%|          | 0/3335 [00:00<?, ?it/s]

Epoch: 1 : 1.678923075095467


  0%|          | 0/3335 [00:00<?, ?it/s]

...

Epoch: 10 : 1.4757779782322393


  0%|          | 0/3335 [00:00<?, ?it/s]

Epoch: 11 : 1.37389829980916


 61%|██████▏   | 2047/3335 [29:43<18:42,  1.15it/s]


KeyboardInterrupt: 

In [44]:
torch.save(model.state_dict(), 'best_model.pth')

# test

In [35]:
if torch.cuda.is_available():
    model.load_state_dict(torch.load('last_model.pth', map_location='cuda'))
    print('using gpu')
else:
    model.load_state_dict(torch.load('last_model.pth', map_location='cpu'))
    print('using cpu')

using gpu


In [36]:
# define test set
test_set = UXP(testset)

## test fit

In [37]:
u, p, y = test_set[1]

u, p, y

(array([2]), array([49451]), array([0.25], dtype=float32))

In [38]:
u = torch.tensor(u).unsqueeze(0).cuda()
p = torch.tensor(p).unsqueeze(0).cuda()

print(u.shape, p.shape)

with torch.no_grad():
    pr = model(u, p)
    
    print(pr)
    
    print(y)

torch.Size([1, 1]) torch.Size([1, 1])
tensor([[[0.3906]]], device='cuda:0')
[0.25]


## avg mse of test_set

In [39]:
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)

In [41]:
mse_collect = []

for u, p, y in tqdm(test_loader):
    # check if gpu avaliable
    if torch.cuda.is_available():
        u = u.to('cuda')
        p = p.to('cuda')
        y = y.to('cuda')

    # forward
    with torch.no_grad():
        pr = model(u, p)

        # calculate loss
        _loss = loss(pr.squeeze(), y.squeeze())
        
        mse_collect.append(_loss.item())
        
print(np.mean(mse_collect))

100%|██████████| 1612/1612 [00:45<00:00, 35.07it/s]

2.572159428759559


In [42]:
np.sqrt(np.mean(mse_collect))

1.6037953200952917

## Top X Hit

In [56]:
testset.head()

,user_id,product_id,uxp_reorder_ratio,u_total_bought_product,u_avg_bought_p_per_order,uxp_order_ratio,uxp_bought_before,score
18,1,49235,0.750000,70,6.363636,0.272727,1,1.122727
139,2,49451,0.083333,226,15.066667,0.066667,1,0.250000
172,3,49683,0.100000,88,7.333333,0.083333,1,0.283333
189,4,43704,0.333333,18,3.600000,0.200000,1,0.633333
217,5,48775,0.200000,46,9.200000,0.200000,1,0.500000


In [125]:
def getBoughtProduct(in_user):
    return int(testset.iloc[in_user]['product_id'])

In [126]:
getBoughtProduct(0)

49235

In [127]:
def getNotBoughtProduct(in_user, n):
    rated = np.array(testset.iloc[in_user]['product_id'])
    
    maxid = np.max(testset['product_id'])
    
    not_rated = []
    while(len(not_rated) < n):
        x = np.random.randint(maxid)
        if (x not in rated) and (x not in np.array(trainset[trainset['user_id'] == testset.iloc[0]['user_id']]['product_id']).astype(int)):
            not_rated.append(x)

    return np.array(not_rated)

In [128]:
getNotBoughtProduct(0, 99)

array([38283, 30668,  5137,   742,  1878, 18955,  9781, 42200, 23433,
       18808, 45812,   138, 26759, 16083,  2829, 31235, 33702, 37823,
       30864, 13105, 14186, 30367, 32232, 42917,  6286, 31454, 18862,
       33113, 47108,   732,  1709, 28743, 28554, 13828, 16725,  1453,
       21712, 48807, 38437,  4803, 23431,  7969, 12537, 36994,  4904,
       11420, 24962, 10469, 41446, 39584, 10695, 29962, 35264, 10575,
        5891, 15702, 26148,   884, 35652, 35759, 27394, 16229, 13796,
       15345, 37835,  7011, 34978, 22997,  5133,  1993,  1011, 28316,
       49599, 33423,  6511, 26599, 14192, 22458, 30431, 47438, 12872,
       48242, 22593, 47309, 20752, 19416, 17859, 24216, 26018, 45915,
       43499, 10488, 21795, 36410, 17528,  9961, 34452, 31917,   153])

In [129]:
eval_data = []

for i in tqdm(range(len(testset))):
    eval_data.append(np.concatenate([[getBoughtProduct(i)],getNotBoughtProduct(i,99)]))

  0%|          | 37/206209 [00:49<76:43:52,  1.34s/it]


KeyboardInterrupt: 

In [130]:
eval_data = np.array(eval_data)

In [131]:
eval_data.shape

(37, 100)

In [132]:
def batch_predict(idx):
#     u = torch.tensor(np.ones(100, dtype=int) * int(testset.iloc[idx]['user_id'])).unsqueeze(0).cuda()
    u = torch.tensor(np.array([testset.iloc[idx]['user_id']]).astype(int)).unsqueeze(0).cuda()
    p = torch.tensor(eval_data[idx]).unsqueeze(0).cuda()

#     print(u.shape, p.shape)

    with torch.no_grad():
        pr = model(u, p)

#         print(pr.shape)
    return pr.cpu().squeeze().numpy()


In [133]:
batch_predict(0)

array([ -0.6153192 , -12.080273  ,   1.8724751 ,   3.489346  ,
         2.203017  ,   0.21421885,  17.79206   ,  -3.0196357 ,
         2.025055  ,   4.0988107 ,   0.6496409 ,  -6.1599073 ,
         0.3991114 ,   3.9705846 ,   1.6922129 ,   2.897554  ,
        -1.8751551 ,  20.339054  ,   8.163025  ,   5.2982397 ,
        -1.751646  ,  -3.9201198 ,   5.492793  ,   2.239483  ,
        -1.1624345 ,   5.0396686 ,   5.2807126 ,   2.2318447 ,
        -5.493971  ,  -0.63085824,   2.6206203 ,   1.2779226 ,
         4.506474  ,  -1.7428782 ,   6.1334667 ,  -1.3533885 ,
        -2.7622592 ,  -2.5223424 ,  -0.9214913 ,   3.2762756 ,
         2.9632707 ,  -7.7606153 ,  -2.1730545 ,  -3.3743243 ,
         5.3492374 ,   6.832577  ,  -0.92292714,  -2.92709   ,
         6.157913  ,   2.8192828 ,  -0.39485884,  -5.712608  ,
        -0.0882746 ,   4.7047253 ,   2.4073434 ,   1.8486947 ,
         1.3755878 ,   3.8346512 ,  -1.8604146 ,   0.96433634,
        24.49153   ,   0.29418603,  -0.30914783,  -1.69

In [134]:
top_1_hit = 0
top_5_hit = 0

# for each user
for user in tqdm(range(len(eval_data))):
    # predict eval_data
    eval_result = list(batch_predict(user))

    eval_result.sort(reverse=True)

    #top 1 hit
    if(eval_data[user][0] == eval_result[0] ):
        top_1_hit += 1

    top_5_rate = [i for i in eval_result[:5]]
    #top 5 hit
    if(eval_data[user][0] in top_5_rate ):
        top_5_hit += 1

    #print('user {} {}'.format(user,top_1_hit))
    #print('user {} {}'.format(user,top_5_hit))
    #print(eval_result)
#     print('{}/{}'.format(user+1,len(eval_data)))

100%|██████████| 37/37 [00:00<00:00, 2268.12it/s]


In [135]:
#top n hit rate
print('top 1 hit rate : {}'.format(top_1_hit/len(eval_data)) )
print('top 5 hit rate : {}'.format(top_5_hit/len(eval_data)) )

top 1 hit rate : 0.0
top 5 hit rate : 0.0


## now we make predict for Kaggle

In [9]:
kaggle_testdf = pd.read_csv('../test.csv')

In [10]:
kaggle_testdf.head()

,Unnamed: 0,order_id,user_id,u_avg_bought_p_per_order
0,0,2774568,3,7.333333
1,1,329954,4,3.600000
2,2,1528013,6,4.666667
3,3,1376945,11,13.428571
4,4,1356845,12,14.800000


In [11]:
len(kaggle_testdf)

75000

In [48]:
# write to file
f = open('predict.txt', 'w')

f.write('order_id,products\n')

18

In [49]:
for row in tqdm(range(len(kaggle_testdf))):
# use to store user scores
    product_scores = {}

    #========================================================================
    # make data

    # predict how many items for this user
    b_p_o = np.round(kaggle_testdf.iloc[row]['u_avg_bought_p_per_order'])

    # user to predict
    u_stack = []
    u = int(kaggle_testdf.iloc[row]['user_id'])
    # stack user to (N_PRODUCT, 1)
    for _p in range(1, N_PRODUCT+1):
        u_stack.append(u)
    u = torch.tensor(np.expand_dims(u_stack, -1))

    # product
    # stack product to (N_PRODUCT, 1)
    p_stack = []
    for _p in range(1, N_PRODUCT+1):
        p_stack.append(int(_p)) 
    p = torch.tensor(np.expand_dims(p_stack, -1))

    # check if gpu avaliable
    if torch.cuda.is_available():
        p = p.to('cuda')
        u = u.to('cuda')

    #==================================================================   
    # forward
    with torch.no_grad():
        pr = model(u, p)


    #=====================================================================
    # store predicted value to dict
    for _p, uxp_score in enumerate(pr.cpu().numpy().squeeze()):
        product_scores[_p + 1] = uxp_score

    # sort by scores
    product_scores = {k : v for k, v in sorted(product_scores.items(), key=lambda  item: item[1], reverse=True)}

    # get top k
    top_k = list(product_scores.keys())[:int(b_p_o)]

    #=========================================================================
    # print output
    f.write('{},'.format(int(kaggle_testdf.iloc[row]['user_id'])))

    first = True
    for k in top_k:
        # formating reason
        if first:
            first = False
        else:
            f.write(' ')

        f.write(str(k))

    f.write('\n')
    
# close file
f.close()

100%|██████████| 75000/75000 [1:30:52<00:00, 13.76it/s]


fix output

In [12]:
submit = {}

In [13]:
from tqdm import tqdm

In [14]:
f = open('predict.txt')

outputs = f.readlines()

for idx, j in tqdm(enumerate(outputs[1:])):
    submit[int(kaggle_testdf.iloc[idx]['order_id'])] = j.split(',')[-1]

75000it [00:08, 8508.81it/s]


In [18]:
# sort submit
submit = {k : v for k, v in sorted(submit.items(), key=lambda  item: item[0], reverse=False)}

{17: '4082 48567 49656 18623 570 45833 49304\n',
 34: '4082 570 49656 26963 24158 15877 40263 11802\n',
 137: '7032 17380 38592 19699 3833 19269 16161 3026 45705 25700 45397 43401\n',
 182: '25282 7032 49562 1413 36237 20300 4257 44062 16391 30932 27097\n',
 257: '48760 10593 7216 20472 43217 28608 49571 33963 33821 35890 8328 38931 6147\n',
 313: '21151 32481 21458 22668 49062 34962 4082\n',
 353: '49656 39665 47949 17496 39992\n',
 386: '37533 47949 43051 39992 570 5273 11788 24158 33278 33263 15642 13084 3542 34961 20002 19216 27350 32126 26992 24189 38084\n',
 414: '570 49656 37533 24158 14128 27097 7032 29253 49550 18506\n',
 418: '49656 8389 15877 44748 26963 4082 39946 49463 570 12588\n',
 437: '49656 15901 49571 7797 7216 41269 20472 28166 37308 22777 41604 44006\n',
 452: '49656 9994 35890 26963 32176 44748 31203 15877 45397 25383 7032 18623 48279 2857 19269 45833 4082 22773 6028 570 14128 5830 36439 24106\n',
 474: '49472 11099 49255 23442 5093 18771\n',
 492: '33963 37533 49

In [20]:
fo = open('predict_order.csv', 'w')
fo.write('order_id,products\n')

18

In [22]:
outputs = f.readlines()

for idx, j in tqdm(enumerate(range(len(submit)))):
    fo.write('{},'.format(list(submit.keys())[j] ))
    fo.write(list(submit.values())[j] )

fo.close()

75000it [04:18, 290.15it/s]
